In [ ]:
import requests
import pandas as pd
import json
from keys import claves as keys
from datetime import date, timedelta
import psycopg2
key = keys.ver(303)

In [ ]:
url="https://api.weatherbit.io/v2.0/history/daily?lat={0}&lon={1}&start_date={2}&end_date={3}&key={4}"

latitude = 40.776676
longitude = -73.971321

year = 2022
month = 1
day = 1

# Dictionary with dtypes from pandas and sql data types.
dic_pdtosql = {
    'object': 'VARCHAR',
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'bool': 'BIT',
    'datetime64': 'DATETIME',
    'timedelta64': 'INTERVAL',
    'category': 'VARCHAR',
    'string' : 'VARCHAR'
}

dic_types = {
    'clouds': 'float',
    'datetime': 'string',
    'dewpt': 'float',
    'dhi': 'float',
    'dni': 'float',
    'ghi': 'float',
    'max_dhi': 'float',
    'max_dni': 'float',
    'max_ghi': 'float',
    'max_temp': 'float',
    'max_temp_ts': 'float',
    'max_uv': 'float',
    'max_wind_dir': 'float',
    'max_wind_spd': 'float',
    'max_wind_spd_ts': 'float',
    'min_temp': 'float',
    'min_temp_ts': 'float',
    'precip': 'float',
    'precip_gpm': 'float',
    'pres': 'float',
    'revision_status': 'string',
    'rh': 'float',
    'slp': 'float',
    'snow': 'float',
    'snow_depth': 'float',
    'solar_rad': 'float',
    't_dhi': 'float',
    't_dni': 'float',
    't_ghi': 'float',
    't_solar_rad': 'float',
    'temp': 'float',
    'ts': 'float',
    'wind_dir': 'float',
    'wind_gust_spd': 'float',
    'wind_spd': 'float'
}

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
start_date = date(year,month,day) # 2023-04-04
end_date = start_date + timedelta(1) # Request required furter date

full_url = url.format(latitude,longitude,start_date,end_date,key)

In [ ]:
response = requests.get(full_url)
# print(response.text)
if response.status_code == 200:
    print("OK")

In [ ]:
json_data = json.loads(response.text, parse_constant=lambda x: x.replace("'", '"'))
df = pd.json_normalize(json_data, record_path =['data'])
df = df.astype(dic_types)

In [ ]:
# Conection to the database.
def conection():
    conn = psycopg2.connect(
        host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com",
        database="data-engineer-database",
        user= keys.ver(101),
        password= keys.ver(202),
        port="5439")
    return conn
conn = conection()
cursor = conn.cursor()

In [ ]:
# df
# for column in df.columns:
#     print(column, df[column].dtype)

In [ ]:
with cursor:
    cursor.execute("DROP TABLE IF EXISTS daily_weather")
cursor = conn.cursor()
# Table creation.
table_name = "daily_weather"
columns = []
for column in df.columns:
    data_type = dic_pdtosql[str(df[column].dtype)]
    columns_sql = '{} {}'.format(column, data_type)
    columns.append(columns_sql)

make_table = "CREATE TABLE {} ({})".format(table_name, ", ".join(columns))

with cursor:
    cursor.execute(make_table)

In [ ]:
row = df.iloc[0]
column = ', '.join(row.index)
values = ', '.join(['%({0})s'.format(col) for col in row.index])

In [ ]:
# cursor = conn.cursor()
# query = "INSERT INTO {} ({}) VALUES ({})".format(table_name, column, values)
# print(query)
# with cursor:
#     cursor.execute(query, row)
# conn.commit()